# MICROSOFT'S MOVIE STUDIO

## BUSINESS UNDERSTANDING

### Background
Microsoft is launching a new movie studio, however they do not know much about the movie industry.

### Business objective
The task is to explore the kind of films that are currently doing the best at the box office and use my findings to give them a better understanding of what kind of films they should get involved in creating.

### Business success criteria
My findings will translate to actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.



## DATA UNDERSTANDING

### Data collection
The datasets below will be used in the analysis:
* `bom.movie_gross.csv.gz` from Box Office Mojo
* `im.db.zip` from IMDB

### Data description




In [2]:
#import the necessary libraries
import pandas as pd #for data analysis and manipulation

import numpy as np #for numerical operations

import sqlite3 # for sqlite database

import seaborn as sns # for data visualization
import matplotlib.pyplot as plt # for data visualization
%matplotlib inline

#### Box Office Mojo Dataset

In [3]:
#load the bom.movie_gross.csv.gz and preview
bom_df = pd.read_csv(r"C:\Users\faith\Documents\Flatiron\dsc-phase-1-project\zippedData\bom.movie_gross.csv.gz")
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
bom_df.tail()

,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [5]:
#summary statistics for numerical columns in our database
bom_df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [6]:
#to get a concise summary of the dataframe
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
#to get a tuple showing the number of rows and columns in our dataset
bom_df.shape

(3387, 5)

**Observations:** 
1. The BOM dataframe(bom_df) has `5` columns:
  * title
  * studio
  * domestic gross
  * foreign gross
  * year
  
2. The dataframe has `3387` rows in total.
3. Our dataframe has `missing` values in the following columns:
  * studio
  * domestic gross
  * foreign gross
4. The domestic gross column is a `float` while the foreign gross is an `object`.
5. Year column is in `int` data type

#### IMDB Database

In [12]:
# connect to the database
conn = sqlite3.connect(r"C:\Users\faith\Documents\Flatiron\dsc-phase-1-project\zippedData\im1.db\im.db")

In [13]:
#find the table names
imdb_df = pd.read_sql("""SELECT name FROM sqlite_master WHERE type = 'table';""", conn)
imdb_df

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [14]:
#preview the movie_basics table
pd.read_sql("""
SELECT *
   FROM movie_basics
   LIMIT 5;
""", conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [15]:
#preview the movie_ratings table
pd.read_sql("""
SELECT *
   FROM movie_ratings
   LIMIT 5;
""", conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


**Observations**
1. There are 8 tables in our database.
2. In this study we will mostly consider the `movie_basics` and `movie_ratings` tables.
3. These two tables are linked together using the `movie_id` key.

## DATA CLEANING

In data cleaning, we will check for missing values, duplicate values and outliers and deal with them.
We observed misssing values in our datasets, it is important to take care of them to enable us to have data we can actually work with.

### BOM Dataframe cleaning

In [16]:
#Check for duplicates
bom_df.duplicated().value_counts()

False    3387
dtype: int64

There are no duplicates in this dataframe.

In [17]:
# checking for null/ missing values
bom_df.isna().any()

title             False
studio             True
domestic_gross     True
foreign_gross      True
year              False
dtype: bool

The following columns have missing values:
 * studio
 * domestic_gross
 * foreign_gross

In [18]:
# check the number of missing values in the columns
bom_df.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [19]:
#locates the rows with null values in the studio column
bom_df.loc[bom_df["studio"].isnull()]

,title,studio,domestic_gross,foreign_gross,year
210,Outside the Law (Hors-la-loi),NaN,96900.0,3300000,2010
555,Fireflies in the Garden,NaN,70600.0,3300000,2011
933,Keith Lemon: The Film,NaN,NaN,4000000,2012
1862,Plot for Peace,NaN,7100.0,NaN,2014
2825,Secret Superstar,NaN,NaN,122000000,2017


In [20]:
# keep the missing values in studio by replacing with string "NaN"
bom_df["studio"] = bom_df["studio"].fillna("NaN")

In [21]:
#check that the change has been implemented
bom_df.isna().sum()

title                0
studio               0
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [25]:
#find the mean and median for the domectic_gross column
print(f"The mean for the column: {bom_df.domestic_gross.mean()}")
print(f"The median for the column: {bom_df.domestic_gross.median()}")

The mean for the column: 28745845.06698422
The median for the column: 1400000.0


The mean is higher than the median which means then distribution is positively skewed.
I will replace the missing values with the mean of the distribution.

In [28]:
bom_df["domestic_gross"] = bom_df["domestic_gross"].fillna(bom_df["domestic_gross"].mean())
bom_df.isna().sum()

title                0
studio               0
domestic_gross       0
foreign_gross     1350
year                 0
dtype: int64

Foreign_gross represents currency, therefore I will change the data type to float.